#### LICENSE

These notes are released under the “Creative Commons
Attribution-ShareAlike 4.0 International” license. See the
**human-readable version**
[here](https://creativecommons.org/licenses/by-sa/4.0/) and the **real
thing**
[here](https://creativecommons.org/licenses/by-sa/4.0/legalcode).

## Some "features" of `R` that are worth remembering

### Lazy evaluations

Arguments passed to function are not "used" (evaluated) until it is
necessary to do so. So they may be ignored in the body of the function
to which they are passed. They can also make it work when calling
a function with an
argument that does not exist. This is a problem because if one 
calls a function with a argument that does not exist, it may 
indicate a logical issue that may never be detected. For example, 
consider the following short example. Consider the (bad!) function 
`f1` (where argument `a` is not used, and the returned object 
`x` does not exist within the function. 

In [ ]:
f1 <- function(a) return(x)

We now create an object `x`  with value `3`
(most of us have an `x` in our `R` environment / frame),
and call `f1(u)`. Recall that there is no `u` when we 
call `f1`. What do you think will happen? 

In [ ]:
x <- 3
f1(u)

Not only calling `f1` with a nonexisting argument works, but it 
also returns the value of an object `x` that does not exist
within `f1` (it is neither defined inside, nor passed as an
argument). 

And it can be more interesting. Consider this change in `f1`

In [ ]:
f1 <- function(a) return(x)/sum(a)

You may think that `f1(u)` will now fail because it will try to evaluate
`a`. But you will be wrong. 

In [ ]:
f1(u)

### Scope

To illustrate how `R` manages its environments, we will call `f1` from within another function `f2`. Again,
its argument `z` is not used. There is an object `x` created inside
`f2`, and then we return the value of `f1(z2)`, which is also suspicious,
since there is no `z2` object anywhere. 

In [ ]:
f2 <- function(z) {
  x <- 7
  return(f1(z2))
}

Will `f2(e)` work? And if it does, what will it return? Which `x` will `f1` use? 

In [ ]:
f2(e)

We now create another function `ff2`, again fairly untidy, but instead of returning `f1(z2)` we
define a function `ff1` inside `ff2`, and return `ff1(z2)`. Again, `z2` does not exist anywhere, 
there again are two possible `x`'s that `ff1` may use. 

In [ ]:
ff2 <- function(u) {
  ff1 <- function(a) return(x)
  x <- 7
  return(ff1(z2))/u^2
}

What would calling `ff2(some.object.that.does.not.exist)` return?

In [ ]:
ff2(some.object.that.does.not.exist)

Discuss. Now `ff1` uses the `x` of the environment that called it, unlike what `f1` did when we called it from within `f2`. What changed? Hint: function environments are attached when they are created. There are ways to override this behaviour explicitly, which is much better coding practice than relying on the default behaviour. 

### Recycling

Consider a matrix `x` with 6 rows and two columns, and a vector `a` with 2 elements. What is `a*x`? is it different from `x*a`? 

In [ ]:
x <- matrix(rnorm(12), 6, 2)
a <- c(3, 4)
a * x

Is it different from `x*a`? 

In [ ]:
x * a

Is `x*a` different from `x %*% a`? 

In [ ]:
x %*% a

Now, create a vector `y` of length 6 and compute `y - a * x` (a vector of length 6 minus a matrix of size 3 by 2). Could this work? yes!

In [ ]:
y <- rnorm(6)
y - a * x

Not only it works, it does it without a warning. But what is the result? 

Note that it also works if `y` has 4 or 12 elements:

In [ ]:
y <- rnorm(4)
y - a * x
y <- rnorm(12)
y - a * x

Note that this is not very unusual. Consider `X` a regression design matrix of size `n` by `d`, and `y` a vector of responses of length `d`, and `beta` the vector of regression coefficients, then 
```
y - X * beta
```
will not give you the residuals you think it may. 

Here is a simple example:

In [ ]:
data(Animals, package='MASS')
fit <- lm(brain ~ body, data=Animals)
beta.hat <- coef(fit)
X <- model.matrix(fit)
y <- Animals$brain

There are 28 data points, so, there should be 28 residuals. 

But:

In [ ]:
cbind(wrong = as.vector( y - X * beta.hat ), real=resid(fit))

However:

In [ ]:
cbind(right = as.vector( y - X %*% beta.hat ), real=resid(fit))